In [14]:
import json
import requests
import pandas as pd
from requests_html import HTMLSession

In [33]:
url = "https://lpff.es/partido/villareal-cf-femenino/fc-barcelona-femenino/2023309307/eventos"
data = requests.get(url)
print(data.content)
json_object = json.loads(s=data.content)



b'<!DOCTYPE html>\n<html lang="es">\n<head>\n\t<meta charset="utf-8">\n\t<meta http-equiv="content-language" content="es">\n\t<meta name="language" content="Spanish">\n\t<title>Liga Profesional de F\xc3\xbatbol Femenino - Inicio</title>\n\t<meta name="theme-color" content="#00256a">\n\t<meta http-equiv="Content-Style-Type" content="text/css" />\n\t<meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1" />\n\t<meta name="description" content="P\xc3\xa1gina web oficial de la Liga Profesional de f\xc3\xbatbol femenino (LPFF). Informaci\xc3\xb3n, noticias, resultados en directo y clasificaciones de todas las competiciones, equipos y jugadores.">\n\t<meta name="keywords" content="--------">\n\t\t<meta property="og:image" content="http://ligaf.es/media/img/graphics/logo-futbolfemenino-og.jpg">\n\t \n\t<meta name="robots" content="index, follow">\n\t\n\t\t<link rel="apple-touch-icon" sizes="57x57" href="/media/img/graphics/favicon/apple-icon-57x57.png">\n\t<link r

JSONDecodeError: Expecting value: line 1 column 1 (char 0)